# Spark Architecture: Understanding Executors, Cores, and Tasks

## Learning Objectives

By the end of this notebook, you will understand:

1. **What Spark architecture is** and why it matters
2. **Executors** - what they are and their role in distributed computing
3. **Cores** - how CPU cores enable parallelism
4. **Tasks** - the unit of work in Spark
5. **How these components work together** to process data in parallel
6. **Basic configuration concepts** for optimizing Spark jobs

## Prerequisites

- Basic understanding of Python
- Familiarity with data processing concepts
- No prior Spark experience required - this is a foundational module!

---

> **Note:** This notebook provides the foundational understanding needed before diving into advanced topics like partitions, optimization, and performance tuning.


## Introduction: Why Understanding Architecture Matters

As a fresh graduate entering the world of big data, you'll often hear terms like:
- "We need more executors"
- "The job is using only 25% of available cores"
- "Tasks are not being distributed evenly"

**Understanding Spark architecture is crucial because:**
- It helps you write efficient code
- It enables you to debug performance issues
- It allows you to optimize resource usage (which saves money!)
- It makes you a better data engineer

**Think of it this way:**
- Without understanding architecture → You're driving blindfolded
- With understanding architecture → You can see the road and make informed decisions

Let's start with the big picture!


## The Big Picture: Spark Cluster Architecture

### High-Level Overview

When you run a Spark application, it runs on a **cluster** - a collection of machines working together. Here's the basic structure:

```
┌─────────────────────────────────────────────────────────────┐
│                    SPARK CLUSTER                             │
│                                                              │
│  ┌──────────────────┐                                       │
│  │   Driver Node    │  ← Your code runs here               │
│  │  (Master/Client) │     (coordinates everything)         │
│  └──────────────────┘                                       │
│           │                                                  │
│           │ Coordinates                                      │
│           │                                                  │
│  ┌──────────────────────────────────────────────┐          │
│  │         Worker Nodes (Executors)              │          │
│  │                                                │          │
│  │  ┌──────────┐  ┌──────────┐  ┌──────────┐   │          │
│  │  │Executor 1│  │Executor 2│  │Executor 3│   │          │
│  │  │          │  │          │  │          │   │          │
│  │  │ 4 Cores  │  │ 4 Cores  │  │ 4 Cores  │   │          │
│  │  │          │  │          │  │          │   │          │
│  │  └──────────┘  └──────────┘  └──────────┘   │          │
│  │                                                │          │
│  └──────────────────────────────────────────────┘          │
│                                                              │
└─────────────────────────────────────────────────────────────┘
```

### Key Components

1. **Driver Node**: 
   - Runs your main program
   - Coordinates the entire job
   - Manages task scheduling

2. **Worker Nodes**: 
   - Physical machines in the cluster
   - Run executors (JVM processes)
   - Do the actual data processing

3. **Executors**: 
   - JVM processes running on worker nodes
   - Execute tasks assigned by the driver
   - Store data in memory/disk

**Simple Analogy:**
- **Driver** = Project manager (coordinates work)
- **Worker Nodes** = Office buildings
- **Executors** = Teams working in those buildings
- **Cores** = Individual workers in each team
- **Tasks** = Specific jobs assigned to workers

## Component 1: Executors

### What is an Executor?

An **executor** is a JVM (Java Virtual Machine) process that runs on a worker node. Think of it as a "worker process" that does the actual data processing.

### Key Characteristics

**1. Process Isolation**
- Each executor is a separate JVM process
- If one executor crashes, others continue working
- Provides fault tolerance

**2. Memory Management**
- Each executor has its own memory space
- Memory is divided into:
  - **Execution memory**: For computations (shuffles, joins, aggregations)
  - **Storage memory**: For caching DataFrames

**3. Task Execution**
- Executors receive tasks from the driver
- Execute tasks in parallel using multiple cores
- Return results back to the driver

### Visual Representation

```
Worker Node (Physical Machine)
┌─────────────────────────────────────┐
│  Operating System                    │
│                                      │
│  ┌────────────────────────────────┐ │
│  │  Executor (JVM Process)        │ │
│  │  ┌──────────┐  ┌──────────┐   │ │
│  │  │  Core 1  │  │  Core 2  │   │ │
│  │  │  [Task]  │  │  [Task]  │   │ │
│  │  └──────────┘  └──────────┘   │ │
│  │  ┌──────────┐  ┌──────────┐   │ │
│  │  │  Core 3  │  │  Core 4  │   │ │
│  │  │  [Task]  │  │  [Task]  │   │ │
│  │  └──────────┘  └──────────┘   │ │
│  │                                │ │
│  │  Memory: 8 GB                  │ │
│  └────────────────────────────────┘ │
└─────────────────────────────────────┘
```

### Common Configuration

```python
# Typical executor configuration
num_executors = 4          # 4 executor processes
executor_cores = 4         # 4 CPU cores per executor
executor_memory = "8g"     # 8 GB RAM per executor
```

**What this means:**
- You have 4 separate JVM processes
- Each process can use 4 CPU cores simultaneously
- Each process has 8 GB of memory
- Total: 16 cores and 32 GB RAM across all executors


## Component 2: Cores

### What is a Core?

A **core** is a CPU core - a physical processing unit that can execute instructions. Modern CPUs have multiple cores, allowing parallel execution.

### Key Concepts

**1. Parallelism = Number of Cores**
- Each core can execute one task at a time
- More cores = more tasks can run simultaneously
- This is your **parallelism capacity**

**2. Core vs Thread**
- **Core**: Physical hardware unit
- **Thread**: Software execution unit
- In Spark: 1 core = 1 thread = 1 task at a time

**3. Total Parallelism**
```
Total Parallelism = Number of Executors × Cores per Executor

Example:
- 4 executors × 4 cores = 16 total cores
- Maximum 16 tasks can run simultaneously
```

### Visual Example

```
Executor with 4 Cores:

┌─────────────────────────────────┐
│  Executor                       │
│                                 │
│  Core 1: [Task A] ← Processing │
│  Core 2: [Task B] ← Processing │
│  Core 3: [Task C] ← Processing │
│  Core 4: [Task D] ← Processing │
│                                 │
│  All 4 cores working in parallel│
└─────────────────────────────────┘
```

### Important Rule: One Task Per Core

**Critical Understanding:**
- ❌ One core cannot process multiple tasks simultaneously
- ❌ Multiple cores cannot share one task
- ✅ One core processes one task at a time
- ✅ Multiple cores can process multiple tasks in parallel

**Why this matters:**
If you have 16 cores but only 4 tasks, you're wasting 12 cores (75% idle)!

```
16 Cores Available:
[●] [●] [●] [●] [○] [○] [○] [○] [○] [○] [○] [○] [○] [○] [○] [○]
 ↑   ↑   ↑   ↑
Used Used Used Used
(4 tasks running, 12 cores idle - BAD!)
```


## Component 3: Tasks

### What is a Task?

A **task** is the smallest unit of work in Spark. It represents processing one partition of data on one executor using one core.

### Key Characteristics

**1. Task = Partition Processing**
- Each task processes exactly one partition
- One partition = one task = one core
- Tasks are independent and can run in parallel

**2. Task Lifecycle**

```
Task Creation → Task Scheduling → Task Execution → Result Return
     ↓              ↓                  ↓              ↓
  Driver        Driver assigns    Executor runs   Results sent
  creates       to executor       on one core     back to driver
```

**3. Task Distribution**

The driver schedules tasks across executors and cores:

```
Driver's Task Queue:
[Task 1] [Task 2] [Task 3] [Task 4] [Task 5] [Task 6] ... [Task 16]

Distribution:
Executor 1 (4 cores): Task 1, Task 2, Task 3, Task 4
Executor 2 (4 cores): Task 5, Task 6, Task 7, Task 8
Executor 3 (4 cores): Task 9, Task 10, Task 11, Task 12
Executor 4 (4 cores): Task 13, Task 14, Task 15, Task 16
```

### Visual Example

```
Cluster with 4 Executors, 4 Cores Each (16 total cores):

Executor 1:          Executor 2:          Executor 3:          Executor 4:
[Task 1]            [Task 5]            [Task 9]            [Task 13]
[Task 2]            [Task 6]            [Task 10]           [Task 14]
[Task 3]            [Task 7]            [Task 11]           [Task 15]
[Task 4]            [Task 8]            [Task 12]           [Task 16]

All 16 tasks running in parallel = All 16 cores utilized = OPTIMAL!
```

### Task vs Partition

**Important Distinction:**
- **Partition**: A logical division of data
- **Task**: The work unit that processes a partition

**Relationship:**
- 1 partition → 1 task (in most cases)
- If you have 100 partitions, you'll have 100 tasks
- If you have 16 cores, you can process 16 tasks simultaneously
- Remaining 84 tasks will wait in queue


## How They Work Together: A Complete Example

### Scenario Setup

Let's say you have:
- **4 executors**
- **4 cores per executor** (16 total cores)
- **100 partitions** of data to process

### Step-by-Step Execution

**Step 1: Driver Creates Tasks**
```
100 partitions → 100 tasks created
```

**Step 2: Driver Schedules Tasks**
```
First batch: 16 tasks (one per core)
Remaining: 84 tasks wait in queue
```

**Step 3: Executors Execute Tasks**
```
Executor 1: [Task 1] [Task 2] [Task 3] [Task 4] → 4 cores busy
Executor 2: [Task 5] [Task 6] [Task 7] [Task 8] → 4 cores busy
Executor 3: [Task 9] [Task 10] [Task 11] [Task 12] → 4 cores busy
Executor 4: [Task 13] [Task 14] [Task 15] [Task 16] → 4 cores busy

Queue: [Task 17] ... [Task 100] (84 tasks waiting)
```

**Step 4: Tasks Complete, New Tasks Start**
```
When Task 1 finishes → Task 17 starts immediately
When Task 2 finishes → Task 18 starts immediately
... and so on until all 100 tasks complete
```

### Timeline Visualization

```
Time →
┌─────────────────────────────────────────────────────────────┐
│ Batch 1: Tasks 1-16 running (16 cores utilized)            │
│ Batch 2: Tasks 17-32 running (16 cores utilized)            │
│ Batch 3: Tasks 33-48 running (16 cores utilized)           │
│ ...                                                          │
│ Batch 7: Tasks 97-100 running (4 cores utilized, 12 idle)   │
└─────────────────────────────────────────────────────────────┘
```

**Key Insight:**
- With 100 partitions and 16 cores, you need ~7 batches
- Each batch processes 16 tasks in parallel
- Last batch has only 4 tasks (12 cores idle, but that's okay)


## Common Scenarios and What They Mean

### Scenario 1: Too Few Partitions (Underutilization)

**Configuration:**
- 4 executors × 4 cores = 16 cores available
- Data has only 4 partitions

**What Happens:**
```
Executor 1: [Task 1] [○] [○] [○]  → 1 core used, 3 idle
Executor 2: [Task 2] [○] [○] [○]  → 1 core used, 3 idle
Executor 3: [Task 3] [○] [○] [○]  → 1 core used, 3 idle
Executor 4: [Task 4] [○] [○] [○]  → 1 core used, 3 idle

Utilization: 4/16 = 25% (75% waste!)
```

**Problem:** You're paying for 16 cores but only using 4!

**Solution:** Increase partitions (e.g., use `repartition(16)` or more)

### Scenario 2: Too Many Partitions (Overhead)

**Configuration:**
- 4 executors × 4 cores = 16 cores available
- Data has 10,000 partitions

**What Happens:**
```
- 16 tasks run simultaneously (good!)
- But 9,984 tasks wait in queue
- Each task is very small (overhead dominates)
- Task scheduling overhead becomes significant
```

**Problem:** Too much overhead, small tasks are inefficient

**Solution:** Reduce partitions to a reasonable number (e.g., 2-4× number of cores)

### Scenario 3: Optimal Configuration

**Configuration:**
- 4 executors × 4 cores = 16 cores available
- Data has 32-64 partitions (2-4× number of cores)

**What Happens:**
```
- All 16 cores stay busy
- Good load balancing
- Reasonable task sizes
- Efficient resource utilization
```

**Result:** Optimal performance! ✅


## Key Takeaways

### The Golden Rules

1. **One Task = One Partition = One Core**
   - A task processes exactly one partition
   - A task uses exactly one core
   - Multiple tasks can run in parallel on multiple cores

2. **Parallelism = Number of Cores**
   - Maximum parallelism = Total cores in cluster
   - To utilize all cores, you need at least that many partitions/tasks

3. **Optimal Partition Count**
   - Too few partitions → Underutilization (cores sit idle)
   - Too many partitions → Overhead (scheduling overhead dominates)
   - Sweet spot: 2-4× the number of cores

### The Formula

```
Total Cores = Number of Executors × Cores per Executor

Optimal Partitions = 2-4 × Total Cores

Example:
- 4 executors × 4 cores = 16 cores
- Optimal partitions: 32-64 partitions
```

### What This Means for You

**As a data engineer, you should:**
- ✅ Understand your cluster configuration (executors, cores)
- ✅ Monitor task distribution in Spark UI
- ✅ Ensure partitions ≥ cores (ideally 2-4×)
- ✅ Avoid creating too few or too many partitions

**Common Mistakes to Avoid:**
- ❌ Assuming Spark will automatically use all cores
- ❌ Not checking partition count before processing
- ❌ Creating millions of tiny partitions
- ❌ Having fewer partitions than cores


## Practical Example: Checking Your Configuration

Let's see how to check your Spark configuration and understand what you're working with.


In [1]:
# Initialize Spark Session
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("SparkArchitectureDemo") \
    .master("local[*]") \
    .getOrCreate()

print("=" * 70)
print("SPARK SESSION INITIALIZED")
print("=" * 70)
print(f"Spark Version: {spark.version}")
print(f"App Name: {spark.sparkContext.appName}")
print(f"Master: {spark.sparkContext.master}")
print("=" * 70)


26/01/02 22:11:44 WARN Utils: Your hostname, N-MacBookPro-37.local resolves to a loopback address: 127.0.0.1; using 192.168.1.4 instead (on interface en0)
26/01/02 22:11:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/02 22:11:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/02 22:11:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


SPARK SESSION INITIALIZED
Spark Version: 3.5.1
App Name: SparkArchitectureDemo
Master: local[*]


In [2]:
# Check Spark Configuration
# This shows you the actual configuration of your Spark cluster

sc = spark.sparkContext

# Get default parallelism (usually equals number of cores)
default_parallelism = sc.defaultParallelism

# Get Spark configuration
conf = sc.getConf()

# Try to get executor information (may not be available in local mode)
try:
    executor_memory = conf.get("spark.executor.memory", "Not set")
    executor_cores = conf.get("spark.executor.cores", "Not set")
    num_executors = conf.get("spark.executor.instances", "Not set")
except:
    executor_memory = "Not available in local mode"
    executor_cores = "Not available in local mode"
    num_executors = "Not available in local mode"

print("=" * 70)
print("SPARK CLUSTER CONFIGURATION")
print("=" * 70)
print(f"Default Parallelism: {default_parallelism}")
print(f"  → This represents the number of cores available")
print(f"\nExecutor Configuration:")
print(f"  Executor Memory: {executor_memory}")
print(f"  Cores per Executor: {executor_cores}")
print(f"  Number of Executors: {num_executors}")
print("\n" + "=" * 70)
print("NOTE: In local mode, you're running on a single machine.")
print("In a real cluster, you'd see multiple executors across worker nodes.")
print("=" * 70)


SPARK CLUSTER CONFIGURATION
Default Parallelism: 11
  → This represents the number of cores available

Executor Configuration:
  Executor Memory: Not set
  Cores per Executor: Not set
  Number of Executors: Not set

NOTE: In local mode, you're running on a single machine.
In a real cluster, you'd see multiple executors across worker nodes.


In [ ]:
# Example: Understanding Partitions and Tasks

# Create a simple DataFrame
data = [(i, f"Product_{i}", 100.0 + i) for i in range(1000)]
columns = ["id", "product_name", "price"]

df = spark.createDataFrame(data, columns)

# Check number of partitions
# Note: While RDDs are legacy APIs, .rdd.getNumPartitions() is still the standard
# way to get partition count from DataFrames (no direct DataFrame API exists for this)
num_partitions = df.rdd.getNumPartitions()

print("=" * 70)
print("UNDERSTANDING PARTITIONS AND TASKS")
print("=" * 70)
print(f"DataFrame created with {len(data)} rows")
print(f"Number of partitions: {num_partitions}")
print(f"Default parallelism: {default_parallelism}")
print("\n" + "=" * 70)
print("KEY INSIGHT:")
print(f"  - This DataFrame has {num_partitions} partitions")
print(f"  - When you perform an operation, Spark will create {num_partitions} tasks")
print(f"  - With {default_parallelism} cores, {min(default_parallelism, num_partitions)} tasks can run simultaneously")
if num_partitions < default_parallelism:
    print(f"  - ⚠️  WARNING: Only {num_partitions} tasks, but {default_parallelism} cores available!")
    print(f"     This means {default_parallelism - num_partitions} cores will be idle.")
elif num_partitions >= default_parallelism:
    print(f"  - ✅ Good: {num_partitions} tasks can utilize all {default_parallelism} cores")
print("=" * 70)


UNDERSTANDING PARTITIONS AND TASKS
DataFrame created with 1000 rows
Number of partitions: 11
Default parallelism: 11

KEY INSIGHT:
  - This DataFrame has 11 partitions
  - When you perform an operation, Spark will create 11 tasks
  - With 11 cores, 11 tasks can run simultaneously
  - ✅ Good: 11 tasks can utilize all 11 cores


In [4]:
# Example: Repartitioning to Optimize

print("=" * 70)
print("REPARTITIONING EXAMPLE")
print("=" * 70)

# Repartition to match or exceed number of cores
optimal_partitions = default_parallelism * 2  # 2× cores for good load balancing

df_repartitioned = df.repartition(optimal_partitions)
# Note: Using .rdd.getNumPartitions() as it's the standard way to check partition count
new_num_partitions = df_repartitioned.rdd.getNumPartitions()

print(f"Original partitions: {num_partitions}")
print(f"Repartitioned to: {new_num_partitions} partitions")
print(f"Default parallelism (cores): {default_parallelism}")
print(f"\n✅ Now you have {new_num_partitions} partitions")
print(f"   This allows better utilization of {default_parallelism} cores")
print(f"   Multiple batches of tasks can run efficiently")
print("=" * 70)


REPARTITIONING EXAMPLE


Original partitions: 11
Repartitioned to: 22 partitions
Default parallelism (cores): 11

✅ Now you have 22 partitions
   This allows better utilization of 11 cores
   Multiple batches of tasks can run efficiently


In [5]:
# Example: Observing Task Execution

print("=" * 70)
print("OBSERVING TASK EXECUTION")
print("=" * 70)
print("Performing a simple operation to demonstrate task execution...")
print("(In Spark UI, you would see tasks being distributed across executors)")
print()

# Perform an operation that triggers task execution
result = df_repartitioned.groupBy("product_name").count()

# Count the results (this triggers execution)
count = result.count()

print(f"Operation completed!")
print(f"  - Processed {new_num_partitions} partitions")
print(f"  - Created {new_num_partitions} tasks")
print(f"  - Tasks executed across available cores")
print(f"  - Result: {count} unique products")
print("\n💡 TIP: Open Spark UI (usually at http://localhost:4040) to see")
print("   task distribution, executor utilization, and execution timeline")
print("=" * 70)


OBSERVING TASK EXECUTION
Performing a simple operation to demonstrate task execution...
(In Spark UI, you would see tasks being distributed across executors)

Operation completed!
  - Processed 22 partitions
  - Created 22 tasks
  - Tasks executed across available cores
  - Result: 1000 unique products

💡 TIP: Open Spark UI (usually at http://localhost:4040) to see
   task distribution, executor utilization, and execution timeline


## Summary: The Complete Picture

### Architecture Components Recap

```
┌─────────────────────────────────────────────────────────────┐
│                    SPARK CLUSTER                             │
│                                                              │
│  ┌──────────────────┐                                       │
│  │   Driver Node    │  ← Creates tasks, coordinates work    │
│  └──────────────────┘                                       │
│           │                                                  │
│           │ Schedules tasks                                  │
│           │                                                  │
│  ┌──────────────────────────────────────────────┐          │
│  │         Worker Nodes                          │          │
│  │                                                │          │
│  │  ┌──────────┐  ┌──────────┐  ┌──────────┐   │          │
│  │  │Executor 1│  │Executor 2│  │Executor 3│   │          │
│  │  │          │  │          │  │          │   │          │
│  │  │ Core 1   │  │ Core 1   │  │ Core 1   │   │          │
│  │  │ Core 2   │  │ Core 2   │  │ Core 2   │   │          │
│  │  │ Core 3   │  │ Core 3   │  │ Core 3   │   │          │
│  │  │ Core 4   │  │ Core 4   │  │ Core 4   │   │          │
│  │  │          │  │          │  │          │   │          │
│  │  │[Task]    │  │[Task]    │  │[Task]    │   │          │
│  │  │[Task]    │  │[Task]    │  │[Task]    │   │          │
│  │  │[Task]    │  │[Task]    │  │[Task]    │   │          │
│  │  │[Task]    │  │[Task]    │  │[Task]    │   │          │
│  │  └──────────┘  └──────────┘  └──────────┘   │          │
│  │                                                │          │
│  └──────────────────────────────────────────────┘          │
│                                                              │
└─────────────────────────────────────────────────────────────┘
```

### Key Relationships

1. **Executors** run on worker nodes (JVM processes)
2. **Cores** are CPU cores within each executor
3. **Tasks** are work units that process partitions
4. **Partitions** are logical divisions of data

**The Flow:**
```
Data → Partitions → Tasks → Scheduled to Executors → Executed on Cores
```

### Best Practices

✅ **DO:**
- Understand your cluster configuration
- Ensure partitions ≥ cores (ideally 2-4×)
- Monitor Spark UI for task distribution
- Repartition data when needed

❌ **DON'T:**
- Assume Spark automatically uses all cores
- Create too few partitions (underutilization)
- Create too many partitions (overhead)
- Ignore partition counts

### Next Steps

Now that you understand the basics:
1. Practice checking partition counts in your DataFrames
2. Learn about `repartition()` and `coalesce()` operations
3. Explore Spark UI to see tasks in action
4. Move on to advanced topics like partition optimization



In [6]:
# Clean up
spark.stop()
print("Spark session stopped.")


Spark session stopped.
